# Example: Computing on the Dynex Platform with Python - BQM

Dynex is the world’s only accessible neuromorphic quantum computing cloud for solving real-world problems, at scale.. This example demonstrates how to use the Dynex SDK to use Pyton to compute on the Dynex Platform with Python.

In [1]:
import dynex
import dimod

## Building a Binary Quadratic Model 

Binary quadratic models (BQMs) are problems of the form:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>E</mi>
  <mo stretchy="false">(</mo>
  <mrow data-mjx-texclass="ORD">
    <mi mathvariant="bold">v</mi>
  </mrow>
  <mo mathvariant="bold" stretchy="false">)</mo>
  <mo mathvariant="bold">=</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">a</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">&lt;</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">b</mi>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">,</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">j</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <mi mathvariant="bold">c</mi>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo>&#x2208;</mo>
  <mo fence="false" stretchy="false">{</mo>
  <mo mathvariant="bold">&#x2212;</mo>
  <mn mathvariant="bold">1</mn>
  <mo mathvariant="bold">,</mo>
  <mo mathvariant="bold">+</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
  <mtext mathvariant="bold">&#xA0;or&#xA0;</mtext>
  <mo fence="false" stretchy="false">{</mo>
  <mn mathvariant="bold">0</mn>
  <mo mathvariant="bold">,</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
</math>

where a,b,c are real values.

We need to define a quadratic model (QM) that represents our problem. The simplest way to build a binary quadratic model (BQM) is using dimod’s symbolic variables. For each mathematical variable in your BQM, we define a symbolic
binary variable using dimod. Once the QM is defined, it is stored as a BinaryQuadraticModel object. This object stores the linear and quadratic coefficients of the mathematical expression, any constant term or offset, and
the type of variables used to build the model. In this case, printing out the object bqmmodel that we have
constructed reveals the following:

In [2]:
# Example: Binary Quadratic Model(BQM):
bqmodel = dimod.BinaryQuadraticModel({'x1': 1.0, 'x2': -1.5, 'x3': 2.0}, 
                                     {('x1', 'x2'): 1.0, ('x2', 'x3'): -2.0}, 
                                      0.0, dimod.BINARY)

model = dynex.BQM(bqmodel);
print(bqmodel);

BinaryQuadraticModel({'x1': 1.0, 'x2': -1.5, 'x3': 2.0}, {('x2', 'x1'): 1.0, ('x3', 'x2'): -2.0}, 0.0, 'BINARY')


## Interacting with the Dynex Sampler 

To find the minimum energy state for a QM (the assignment of variable values that gives us the
minimum energy value for our QM), the Dynex SDK provides samplers and solvers. A solver is
a resource that runs a problem. Samplers are processes that run a problem many times to obtain
a collection of samples, each of which is a possible solution to our problem. For convenience, we
will generally refer to Dynex’s samplers as a whole, to include solvers as well.

In [4]:
sampler = dynex.DynexSampler(model, mainnet=False);

Once we have established our sampler in our program, we can call it for our QM. Each type of QM
model has its own method for interacting with the sampler, whether it be QUBO, BinaryQuadrticModel, or any other QM. We call the sampler to sample our QM using one of Dynex’s sample functions, depending on what type of QM we are using. For example, the code snippet below demonstrates how we can sample a BinaryQuadraticModel object named bqm using the Dynex Platform.

In [5]:
sampleset = sampler.sample(num_reads=32, annealing_time = 10);

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │       11 │              15 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.32 SECONDS
[DYNEX] SAMPLESET READY


After we have sampled our QM, the sampler returns a SampleSet object. This object contains all
of the samples returned along with their corresponding energy value, number of chips, number of integration steps, and more. The additional information varies depending on which sampler is used. As users get more
comfortable with the Dynex SDK and the variety of samplers available, it is often useful to take
some time to explore the wealth of information provided in the SampleSet object. Some of the key properties and methods of a SampleSet that we access are the following:

In [6]:
print(sampleset.first) # sample with the lowest energy

Sample(sample={'x1': 0, 'x2': 1, 'x3': 1}, energy=-1.5, num_occurrences=1)


The sample shows the corresponding energy values for the variables X1, X2 and X3 of our QM and additional information like total energy, number of chips or number of integration steps.

In [7]:
sampleset # the full set of samples

SampleSet(rec.array([([0, 1, 1], -1.5, 1)],
          dtype=[('sample', 'i1', (3,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(['x1', 'x2', 'x3']), {}, 'BINARY')

The sample shows the corresponding energy values for our sample and additional information like total energy, number of occurences or type of variables.